In [ ]:
import numpy as np
import numexpr as ne
import matplotlib.pyplot as plt
import tqdm

In [ ]:
# Relationship for "special" FFT
# For both the individual tiles *and* the whole thing, it is true that
# the second half is just the reversed conjugate of the first half.
# In retrospect that shouldn't be a surprise!
a = np.array([1, 0, 0, 0, 3.4, 0, 0, 0, 2.19, 0, 0, 0, 0.765, 0, 0, 0])
print(np.fft.rfft(a))

In [ ]:
# Check that my code works (for a 2d array)
from numpy.fft import fft, fftn, rfft, rfftn, irfftn
Nnum = 4
a = np.random.random((8*Nnum,8*Nnum))
subset = np.zeros(a.shape)
subset[::Nnum, ::Nnum] = a[::Nnum, ::Nnum]
longhandResult = fftn(subset)

reduced = a[::Nnum, ::Nnum]
reducedShape = reduced.shape
fshape = a.shape
reducedF = fftn(reduced, reducedShape)
result = np.tile(reducedF, (1,Nnum))   # edited
#result *= np.exp(-1j * aa * 2*np.pi / fshape[1] * np.arange(result.shape[1]))
result = np.tile(result, (Nnum,1))
#result *= np.exp(-1j * bb * 2*np.pi / fshape[0] * np.arange(result.shape[0]))[:,np.newaxis]

print(np.max(np.abs(result - longhandResult)))

In [ ]:
a = np.pad(np.random.random(16), (0,0), 'constant')
print(a)
print(a[::-1])

In [ ]:
# Relationship between the FT of a and of the reversed version of a
af = np.fft.fft(a)
afm = np.fft.fft(a[::-1])
af = af.conj() * np.exp(1j * 2*np.pi / a.shape[0] * np.arange(a.shape[0]))
print(np.max(np.abs(af - afm)))

In [ ]:
# Relationship between the FT of a and of the reversed version of a when you take into account padding
padLength = 14
af = np.fft.fft(np.pad(a, (0,padLength), 'constant'))
afm = np.fft.fft(np.pad(a[::-1], (0,padLength), 'constant'))
afm = afm.conj() * np.exp(1j * (1+padLength) * 2*np.pi / af.shape[0] * np.arange(af.shape[0]))
print(np.max(np.abs(af - afm)))

In [ ]:
# Relationship with reversed version (with padding) in 2D
# The 2D version requires even more gymnastics:
Hts = np.random.random((3,3))
fshape = (7,10)

fHtsFull = fftn(Hts, fshape)
fHtsFull2 = fftn(Hts[:,::-1], fshape)
padLength = fshape[1] - Hts.shape[1]
fHtsFull = fHtsFull.conj() * np.exp(1j * (1+padLength) * 2*np.pi / fshape[1] * np.arange(fshape[1]))
fHtsFull[1::] = fHtsFull[1::][::-1]
print(np.max(np.abs(fHtsFull-fHtsFull2)))

fHtsFull = fftn(Hts, fshape)
fHtsFull2 = fftn(Hts[::-1,:], fshape)
padLength = fshape[0] - Hts.shape[0]
fHtsFull = fHtsFull.conj() * np.exp(1j * (1+padLength) * 2*np.pi / fshape[0] * np.arange(fshape[0])[:,np.newaxis])
fHtsFull[:,1::] = fHtsFull[:,1::][:,::-1]
print(np.max(np.abs(fHtsFull-fHtsFull2)))

# But why did I need to reverse the full array in the 2D case?
# I don't see the logic behind that. Is there a more appropriate (and faster) transformation I can perform?
# -> We can factor out the exp term and multiply it at the end.
#    What is then left comes down to the relationship between FT(x) and FT(x*), which is the reversal

In [ ]:
# Relationship between the FT of a and the transposed version of a when you take into account padding
b = np.random.random((16,16))
bf = np.fft.fft2(np.pad(b, ((0,padLength),(0,padLength)), 'constant'))
bft = np.fft.fft2(np.pad(b.transpose(), ((0,padLength),(0,padLength)), 'constant'))
print(np.max(np.abs(bf - bft.transpose())))

In [ ]:
# Relationship with transpose when working with non-square padding(!?)
# -> Sadly I don't think there is any useful result here.
# My reason is based on the fact that there is not (as far as I know) any computationally-simple relationship
# between an FFT with one amount of padding and an FFT with another amount of padding.
# That means that the fft of the transpose of an array looks fundamentally different,
# when the same rectangular padding is used in both cases.

In [ ]:
# Work out some code that covers the whole subaperture using mirror/transpose operations
# to minimize the number of times we actually need to compute the FFT of the PSF matrix.

Nnum = 7

covered = np.zeros((Nnum,Nnum))

def process(aa,bb):
    covered[aa,bb] += 1
    
def processFor2(aa,bb,Nnum,mirrorX):
    process(aa,bb)
    if mirrorX:
        process(aa,Nnum-bb-1)

def processFor(aa,bb,Nnum,mirrorX,mirrorY):
    processFor2(aa,bb,Nnum,mirrorX)
    if mirrorY:
        processFor2(Nnum-aa-1,bb,Nnum,mirrorX)
        
    
for aa in range(int((Nnum+1)/2)):
    for bb in range(aa,int((Nnum+1)/2)):
        cent = int(Nnum/2)
        transpose = ((aa != bb) and (aa != (Nnum-bb-1)))
        mirrorX = (bb != cent)
        mirrorY = (aa != cent)
        processFor(aa,bb,Nnum,mirrorX,mirrorY)
        if transpose:
            processFor(bb,aa,Nnum,mirrorY,mirrorX) # Note that mx,my are swapped after transpose

print(covered)